In [ ]:
from openai import OpenAI
import os
import pandas as pd

# Load the main product CSV (which contains 'metacategory')
products_df = pd.read_csv('results/top_products_report.csv')

# Check available columns for debugging
print("Product DataFrame columns:", products_df.columns.tolist())

# Confirm the grouping column exists
category_col = 'metacategory'
if category_col not in products_df.columns:
    raise KeyError(f"'{category_col}' column not found in products_df!")

reports = []

# Iterate per category
for category in products_df[category_col].dropna().unique():
    category_df = products_df[products_df[category_col] == category]
    # Choose the score column based on availability
    score_col = 'positive_proba_mean' if 'positive_proba_mean' in category_df.columns else 'positive_proba'
    top_products = category_df.sort_values(score_col, ascending=False).head(3)
    worst_product = category_df.sort_values(score_col, ascending=True).head(1)

    def format_product_row(row):
        review_excerpt = row['review_text'] if 'review_text' in row else ''
        score = row[score_col]
        return (f"{row['product_name']} (Score: {score:.2f})\nSample Review: {review_excerpt}")

    top_products_text = "\n\n".join([format_product_row(row) for _, row in top_products.iterrows()])
    worst_product_text = format_product_row(worst_product.iloc[0])

    user_msg = f"""
    Category: {category}

    Top 3 products:
    {top_products_text}

    Worst product:
    {worst_product_text}

    Please include:
    - The top 3 products and the key differences between them
    - Who should pick each product and why
    - The top complaints for each product from reviews
    - What is the worst product in the category and why readers should avoid it
    - Reference comparison insights from trusted review sites (The Verge, Wirecutter, Consumer Reviews, etc.)
    Your answer should be concise, insightful, and actionable.
    """
    reports.append({"role": "user", "content": user_msg})

# Perplexity API details

API_KEY = os.environ.get("PPLX_API_KEY", PPLX_API_KEY)
PPLX_BASE_URL = "https://api.perplexity.ai"
MODEL_NAME = "sonar-pro"

client = OpenAI(
    api_key=API_KEY,
    base_url=PPLX_BASE_URL
)

system_msg = {
    "role": "system",
    "content": "You are an expert product reviewer writing blog-style comparison articles for consumers. Use insights from The Wirecutter, The Verge, and Consumer Reviews sites."
}

for report in reports:
    messages = [system_msg, report]
    try:
        print(f"--- Generating report for: {report['content'].splitlines()[1]} ---")
        response = client.chat.completions.create(
            model=MODEL_NAME,
            messages=messages,
            temperature=0.7
        )
        print(response.choices[0].message.content)
        print("----------------------")
    except Exception as e:
        print(f"An error occurred: {e}\nThis environment cannot execute external API calls.")


Product DataFrame columns: ['metacategory', 'product_name', 'positive_proba_mean', 'review_count']
--- Generating report for:     Category: Tablets & E-Readers ---
The top 3 products in the Tablets & E-Readers category for 2025 are:

| Product                           | Key Features                                                        | Best For                       |
|------------------------------------|---------------------------------------------------------------------|-------------------------------|
| **Onyx Boox Tab Ultra C Pro**      | Color E Ink, Android 13, stylus, multitasking, Google Play, USB-C   | Power users, note-takers, productivity |
| **Amazon Kindle Paperwhite (2024)**| 6.8" 300ppi E Ink, IPX8 waterproof, fast processor, Audible support | Dedicated readers, audiobook fans      |
| **Onyx Boox Note Air4 C**          | 10.3" color E Ink, stylus, OCR, battery-free stylus, Android 13     | Students, professionals, annotators    |

**Key Differences:**

- **Display